In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from PIL import Image
import os
from skimage import color
from skimage.feature import hog
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier # using 1NN
import re
import torch
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision.utils import make_grid
import requests
from skimage.feature import hog
from skimage import exposure
from io import BytesIO

In [3]:
# If you have a GPU, you can use it for faster training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Device: ", device)

Device:  cuda


# 1. Download the [cow teat datasets](https://github.com/YoushanZhang/SCTL) (10 points) resize image to (224, 224)

### (1). Create a train data loader that returns image arrays and labels

In [4]:
# Define the path for the training data images .
train_data = r'C:\AKA\Backup Dell Laptop\D Drive\YU\Semester 2\Neural Network\DLNN\Assignment_Week5\Homework Week5\Homework Week5\Training'

# Defineing a transformation that we need to apply to all the images for example resize to (224, 224). 
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the image to a fixed size
    transforms.ToTensor()           # Convert the image to a tensor
])

In [5]:
# Create a training dataset using ImageFolder
dataset = ImageFolder(root=train_data, transform=transform)

In [6]:
# Define a function to extract the numerical part from a string
def extract_label(s):
    '''
        This function is used to extract the label of the training image based on the suffix number of Score Folder. 
        
        Input: 
            S : Folder Name like Score_1, Score_2 etc. 
        Output: 
            The numerical part from the folder name like '1', '2' etc. 
    '''
    # Use regular expressions to extract the numerical part of the string
    match = re.search(r'\d+', s)
    if match:
        return int(match.group())
    else:
        return None

In [7]:
# Create a training data loader for the dataset
train_data_loader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)

In [9]:
# Initialize lists to store data
train_image_arrays = []
train_image_names = []
train_labels = []

# Prepare data with Image Array, Image Name, and Label
for index, (images, batch_labels) in enumerate(train_data_loader):
    # Iterate through each batch of images and labels
    for i in range(len(images)):
        image_array = images[i].numpy()  # Convert tensor to NumPy array
        image_name = os.path.basename(dataset.imgs[index * 32 + i][0])
        subfolder_name = os.path.basename(os.path.dirname(dataset.imgs[index * 32 + i][0]))
        numerical_label = extract_label(subfolder_name)
        if numerical_label is not None:
            train_image_arrays.append(image_array)
            train_image_names.append(image_name)
            train_labels.append(numerical_label)

# Create a pandas DataFrame
df_train = pd.DataFrame({'Image Name': train_image_arrays, 'Size': train_image_names, 'Label': train_labels})

### (2). Create a test data loader that returns image arrays and file names

In [18]:
# Define the path for the test data images .
test_data = r'C:\AKA\Backup Dell Laptop\D Drive\YU\Semester 2\Neural Network\DLNN\Assignment_Week5\Homework Week5\Homework Week5\Test'

In [19]:
# Create a dataset using ImageFolder
test_dataset = ImageFolder(root=test_data, transform=transform)

In [20]:
# Create a test data loader for the test dataset
test_data_loader = DataLoader(test_dataset, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)

In [21]:
# Initialize lists to store data
test_image_arrays = []
test_image_names = []
test_labels = []

# Prepare data with Image Array, Image Name, and Label
for index, (images, batch_labels) in enumerate(test_data_loader):
    # Iterate through each batch of images and labels
    for i in range(len(images)):
        image_array = images[i].numpy()  # Convert tensor to NumPy array
        image_name = os.path.basename(dataset.imgs[index * 32 + i][0])
        subfolder_name = os.path.basename(os.path.dirname(dataset.imgs[index * 32 + i][0]))
        numerical_label = extract_label(subfolder_name)
        if numerical_label is not None:
            test_image_arrays.append(image_array)
            test_image_names.append(image_name)
            test_labels.append(numerical_label)

# Create a pandas DataFrame
df_test = pd.DataFrame({'Image Name': test_image_names, 'test_image_arrays': test_image_arrays})

### (3). Print image arrays, labels and file names dimensions 

In [29]:
# Display Training Data Summary
print(f"Number of Training Images: {len(df_train)}")
print(f"Training Image Array Size: {train_image_arrays[1].shape}")  # Assuming all images have the same size
print(f"Training Image Labels Size: {df_train['Label'].shape}")

Number of Training Images: 1149
Training Image Array Size: (3, 224, 224)
Training Image Labels Size: (1149,)


In [31]:
# Display Test Image Summary
print(f"Number of Test Images: {len(df_test)}")
print(f"Test Image Array Size: {test_image_arrays[0].shape}")  # Assuming all images have the same size

Number of Test Images: 380
Test Image Array Size: (3, 224, 224)


# 2. Extract features of training and test images using HOG (20 points)
Please print the size of extracted features, e.g., training features: 1149 * d, test features: 380 *d

SyntaxError: invalid syntax (2818090206.py, line 34)

In [83]:
# Initialize lists to store HOG features
train_hog_features = []
test_hog_features = []

# Iterate through the training data loader
for images, _ in train_data_loader:
    for img in images:
        # Convert PyTorch tensor to NumPy array and convert to grayscale
        img = img.numpy().transpose((1, 2, 0))
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

        # Resize the image to (224, 224) as you did before
        gray_resized_img = cv2.resize(gray_img, (224, 224))

        # Calculate HOG features
        features = hog(gray_resized_img, pixels_per_cell=(16, 16), visualize=False)

        # Append the features to the list
        train_hog_features.append(features)

# Iterate through the test data loader
for images, _ in test_data_loader:
    for img in images:
        img = img.numpy().transpose((1, 2, 0))
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        gray_resized_img = cv2.resize(gray_img, (224, 224))
        features = hog(gray_resized_img, pixels_per_cell=(16, 16), visualize=False)
        test_hog_features.append(features)

# Convert the lists to NumPy arrays
train_hog_features = np.array(train_hog_features)
test_hog_features = np.array(test_hog_features)

# Now, train_hog_features contains HOG features for training data, and test_hog_features contains HOG features for test data.


In [86]:
train_hog_features.shape

(1149, 11664)

In [87]:
test_hog_features.shape

(380, 11664)

In [ ]:
# Set HOG parameters
orientations = 9
pixels_per_cell = (8, 8)
cells_per_block = (3, 3)

# Initialize lists to store features and labels
train_hog_features = []
train_hog_labels = []
test_hog_features = []
test_hog_labels = []

# Extract features for training images
for i, img in enumerate(train_data):
    # Compute HOG descriptors for each image
    features = hog(img, orientations=orientations, pixels_per_cell=pixels_per_cell,
                   cells_per_block=cells_per_block, transform_sqrt=True, feature_vector=True)
    train_hog_features.append(features)
    train_hog_labels.append(labels[i])
    
# Extract features for test images
for i, img in enumerate(test_data):
    # Compute HOG descriptors for each image
    features = hog(img, orientations=orientations, pixels_per_cell=pixels_per_cell,
                   cells_per_block=cells_per_block, transform_sqrt=True, feature_vector=True)
    test_hog_features.append(features)
    test_hog_labels.append(test_file_names[i])

# Convert feature lists to numpy arrays
train_hog_features = np.array(train_hog_features)
test_hog_features = np.array(test_hog_features)

# Print the size of extracted features
print(f"Training features: {train_hog_features.shape[0]} * {train_hog_features.shape[1]}")
print(f"Test features: {test_hog_features.shape[0]} * {test_hog_features.shape[1]}")

# 3. Extract features of training and test images using SIFT (20 points)
Please print the size of extracted features, e.g., training features: 1149 * d, test features: 380 *d

# 4. Extract features of training and test images using SURF (20 points)
Please print the size of extracted features, e.g., training features: 1149 * d, test features: 380 *d

# 5. Call SVM and kNN from scikit-learn and train the extracted HOG, SIFT and SURF features, respectively, save three CSV files of test dataset using three features (10 points)

# 6. Report the accuracy using Cow_teat_classfication_accuracy software, please attach the results image here (20 points)

### (1). SVM and 1NN using HOG features

In [88]:
#train_hog_features = np.array(train_hog_features)
#test_hog_features = np.array(test_hog_features)

In [91]:
train_hog_features.shape

(1149, 11664)

In [90]:
from sklearn import svm
from sklearn.metrics import accuracy_score

# Create an SVM classifier
clf = svm.SVC()

# Train the SVM classifier on the training HOG features and labels
clf.fit(train_hog_features, train_labels)

# Perform predictions on the test HOG features
test_predictions = clf.predict(test_hog_features)

# Calculate accuracy
accuracy = accuracy_score(test_labels, test_predictions)

# Return the accuracy
accuracy


C:\Users\ankit\anaconda3\envs\aka\Lib\site-packages\sklearn\metrics\_classification.py:226: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  score = y_true == y_pred


0.0

In [95]:
import numpy as np
import cv2
from skimage.feature import hog
from sklearn import svm
from sklearn.metrics import accuracy_score
from tqdm import tqdm  # Import tqdm for progress bars

# ... (Previous code for HOG feature extraction)

# Train SVM classifier on HOG features
svmHog = svm.SVC(kernel='linear')
svmHog.fit(train_hog_features, train_labels)

# Predict using SVM classifier
svmHog_preds = []
for features in tqdm(test_hog_features):
    svmHog_preds.append(svmHog.predict([features]))

# Calculate accuracy
accuracy = accuracy_score(test_labels, svmHog_preds)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Convert numpy array to dataframe
svm_preds_df = pd.DataFrame(svmHog_preds)
svm_preds_df = pd.DataFrame(test_file_names, svmSiftPred)

# Write predictions to CSV files
#svm_preds_df.to_csv("csv/hog_test_predictions_svm.csv")
print("HOG predictions saved to CSV files")

svm_preds_df


100%|████████████████████████████████████████████████████████████████████████████████| 380/380 [00:04<00:00, 83.04it/s]

Accuracy: 0.00%
HOG predictions saved to CSV files



C:\Users\ankit\anaconda3\envs\aka\Lib\site-packages\sklearn\metrics\_classification.py:226: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  score = y_true == y_pred


,0
0,Score_2
1,Score_2
2,Score_1
3,Score_1
4,Score_3
...,...
375,Score_2
376,Score_3
377,Score_2
378,Score_2


### (2). SVM and 1NN using SIFT features

### (3). SVM and 1NN using SURF features

### Citations:
   1. https://towardsdatascience.com/beginners-guide-to-loading-image-data-with-pytorch-289c60b7afec 